In [1]:
import numpy as np
import scipy.integrate as integrate
import scipy.special as special

import random
import os
import csv

import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim

import json

from matplotlib import pyplot as plt
from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

$$
\sin(\pi x) e^{-\pi^2 t}
$$

In [2]:
def heat_equ_analytical_solu(x, t):
    return np.sin(np.pi * x) * np.exp(-np.power(np.pi, 2) * t)

In [3]:
# ============
# save and load: json
# ============

def save_json(save_path, data):
    assert save_path.split('.')[-1] == 'json'
    with open(save_path,'w+') as file:
        json.dump(data,file)

def load_json(file_path):
    assert file_path.split('.')[-1] == 'json'
    with open(file_path,'r') as file:
        data = json.load(file)
    return data

# ============
# save and load: csv
# ============

def save_csv(save_path, data):
    with open(save_path, "w+") as f:
        writer = csv.writer(f)
        writer.writerows(data)

def load_csv(file_path):
    string = []
    with open(file_path, 'r') as f:
        reader = csv.reader(f)
        for i, row in enumerate(reader):
            if i%2==0:
                string.append(row)
    ret = []
    for p in string:
        ret.append(np.array([float(i) for i in p]))
    return ret

# ==============
# save and load: normal
# ==============

def save_list(save_path, data):
    file = open(save_path, 'w+')
    for value in data:
        file.write(str(value)+" ")
    file.close()

# def load_list(file_path):

# ==========
# make directory
# ==========

def mkdir(folder_name):
    folder = os.path.exists(folder_name)
    if not folder:
        os.makedirs(folder_name)

In [4]:
# =======
# OPTIONS
# =======

# ===========
# padding method
# ===========

def padding(starting_padding, original, end_padding):
    return np.hstack((starting_padding, original, end_padding)).tolist()

def zero_padding(original, num1, num2):
    starting_padding = [0 for i in range(num1)]
    end_padding = [0 for i in range(num2)]
    return padding(starting_padding, original, end_padding)

def border_padding(original, num1, num2):
    starting = original[0]
    ending = original[len(original)-1]
    starting_padding = [starting for i in range(num1)]
    end_padding = [ending for i in range(num2)]
    return padding(starting_padding, original, end_padding)

def recursive_padding(original, num1, num2):
    starting_padding = original[-num1:]
    end_padding = original[:num2]
    return padding(starting_padding, original, end_padding)

def random_padding(original, num1, num2):
    max_value = np.max(original)
    min_value = np.min(original)
    starting_padding = [random.randint(min_value, max_value) for i in range(num1)]
    end_padding = [random.randint(min_value, max_value) for i in range(num2)]
    return padding(starting_padding, original, end_padding)

In [5]:
a=[1,2,3,4,5,6,7,8,9,10]
recursive_padding(a, 3, 3)

[8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3]

In [6]:
# =======
# OPTIONS
# =======

# ==================
# analytical solution generator
# ==================

def gen_analytical(delta_x, delta_t, xmin, tmin, xmax, tmax, analytical_eq):
    x = arange(xmin, xmax+delta_x, delta_x)
    t = arange(tmin, tmax+delta_t, delta_t)
    X,T = meshgrid(x, t) # grid of point
    solu = analytical_eq(X, T) # evaluation of the function on the grid
    return x, t, solu

def gen_analytical_averaged(delta_x, delta_t, xmin, tmin, xmax, tmax, analytical_eq):
    x = arange(xmin, xmax+delta_x, delta_x)
    t = arange(tmin, tmax+delta_t, delta_t)
    X,T = meshgrid(x, t) # grid of point
    Z = analytical_eq(X, T) # evaluation of the function on the grid
    solu = []
    for zz in Z:
        solu_t = []
        for j in range(len(zz)-1):
            value = (1/2) * (zz[j] + zz[j+1])
            solu_t.append(value)
        solu.append(solu_t)
    return x, t, solu

def gen_analytical_cell_averaged(delta_x, delta_t, xmin, tmin, xmax, tmax, analytical_eq):
    x = arange(xmin, xmax+delta_x, delta_x)
    t = arange(tmin, tmax+delta_t, delta_t)
    solu = []
    for ti in range(len(t)):
        solu_t = []
        for j in range(len(x)-1):
            value = integrate.quad(lambda x: analytical_eq(x, t[ti]), x[j], x[j+1])
            value = value[0] * (1/delta_x)
            solu_t.append(value)
        solu.append(solu_t)
    return x, t, solu

In [7]:
# =======
# OPTIONS
# =======

# ===========
# training set
# ===========

def get_trainingset_random(solu, num1, num2, padding, size):
    solu_padding = []
    pairs = []
    for item in solu:
        p = padding(item, num1, num2)
        solu_padding.append(p)
    for iteration in range(size):
        t_index = random.randint(0, len(solu_padding)-2)
        x_index = random.randint(0, len(p)-num1-num2-1)
        time = solu_padding[t_index]
        time_next = solu_padding[t_index+1]
        train = time[x_index: x_index+num1+num2+1]
        target = time_next[x_index+num1]
        pair = {'train': train, 'target': target}
        pairs.append(pair)
    return pairs
    
def get_trainingset_all(solu, num1, num2, padding):
    solu_cutted = solu[:-1]
    pairs = []
    for index, item in enumerate(solu_cutted):
        p = padding(item, num1, num2)
        p_next = padding(solu[index+1], num1, num2)
        for xi in range(len(p)-num1-num2):
            train = p[xi: xi+num1+num2+1]
            target = p_next[xi+num1]
            pair = {'train': train, 'target': target}
            pairs.append(pair)
    return pairs

# ===========
#  testing set
# ===========

def get_testingset_random(solu, num1, num2, padding, size):
    return get_trainingset_random(solu, num1, num2, padding, size)

def get_testingset_all(solu, num1, num2, padding):
    solu_cutted = solu[:-1]
    pairs = []
    for index, item in enumerate(solu_cutted):
        p = padding(item, num1, num2)
        p_next = padding(solu[index+1], num1, num2)
        pairs_t = []
        for xi in range(len(p)-num1-num2):
            train = p[xi:xi+num1+num2+1]
            target = p_next[xi+num1]
            pair = {'train': train, 'target': target}
            pairs_t.append(pair)
        pairs.append(pairs_t)
    return pairs

# x, t, solu = gen_analytical_cell_averaged(delta_x=1/10, delta_t=1/20, xmin=0, tmin=0, xmax=1, tmax=2, analytical_eq=heat_equ_analytical_solu)
# pairs = get_testingset_all(solu, 3, 3, recursive_padding)
# for i, p in enumerate(pairs[0]):
#     print(i, p)

In [8]:
x, t, solu = gen_analytical_cell_averaged(delta_x=1/10, delta_t=1/20, xmin=0, tmin=0, xmax=2, tmax=1, analytical_eq=heat_equ_analytical_solu)
print(solu[0])
pairs = get_trainingset_all(solu, 3, 3, recursive_padding)
for i, p in enumerate(pairs):
    print(i, p)

[0.15579194727527879, 0.45212584056020866, 0.7042025064251416, 0.8873469244938119, 0.9836316430834657, 0.9836316430834668, 0.8873469244938119, 0.7042025064251412, 0.45212584056020827, 0.155791947275279, -0.15579194727527892, -0.45212584056020916, -0.7042025064251403, -0.8873469244938128, -0.9836316430834647, -0.983631643083467, -0.8873469244938128, -0.7042025064251405, -0.45212584056020905, -0.15579194727527823]
0 {'train': [-0.7042025064251405, -0.45212584056020905, -0.15579194727527823, 0.15579194727527879, 0.45212584056020866, 0.7042025064251416, 0.8873469244938119], 'target': 0.09511067616387091}
1 {'train': [-0.45212584056020905, -0.15579194727527823, 0.15579194727527879, 0.45212584056020866, 0.7042025064251416, 0.8873469244938119, 0.9836316430834657], 'target': 0.27602193283364607}
2 {'train': [-0.15579194727527823, 0.15579194727527879, 0.45212584056020866, 0.7042025064251416, 0.8873469244938119, 0.9836316430834657, 0.9836316430834668], 'target': 0.42991423955977376}
3 {'train': 

In [9]:
# ==============
# FullconnectedResNet
# ==============

class FullconnectedResNet(nn.Module):
    def __init__(self, i, o, layer_data, hasDropout, p, num, weight=1):
        super(FullconnectedResNet, self).__init__()
        self.layers = torch.nn.Sequential()
        self.layers.add_module("linear_1", nn.Linear(i, layer_data[0]))
        if hasDropout:
            self.layers.add_module("dropout_1", nn.Dropout(p=p))
        self.layers.add_module("relu_1", nn.ReLU())
        for index in range(len(layer_data)-1):
            self.layers.add_module("linear_"+str(index+2), nn.Linear(layer_data[index], layer_data[index+1]))
            if hasDropout:
                self.layers.add_module("dropout_2", nn.Dropout(p=p))
            self.layers.add_module("relu_"+str(index+2), nn.ReLU())
        self.layers.add_module("linear_3"+str(len(layer_data)+1), nn.Linear(layer_data[len(layer_data)-1], o))
        if hasDropout:
            self.layers.add_module("dropout_3", nn.Dropout(p=p))
        self.layers.add_module("relu_"+str(len(layer_data)+1), nn.ReLU())
        self.num = num
        self.weight = weight
        
    def forward(self, x):
        output = self.layers(x)
        return output + self.weight * x[self.num], x[self.num], output

    def load_model(self, save_path):
        self.load_state_dict(torch.load(save_path))

    def save_model(self, save_path):
        torch.save(self.state_dict(), save_path)
        
# ==========
# BidirectionRNN
# ==========

# class BidirectionRNN(nn.Module):

In [10]:
def printer_of_nn(counter, train, target, loss, output, res, pure, TorE):
    print("="*60)
    print("This is", counter+1, "times of iteration")
    if TorE:
        print("Training...")
    else:
        print("Evaluating...")
    print("The input is:", train)
    print("The output is:", output)
    print("The correct solution is:", target)
    print("The pure output is:", pure)
    print("The res input is:", res)
    if TorE:
        print("The training loss is:", loss)
    else:
        print("The evaluation loss is:", loss)

In [11]:
def json_config_generator(
    delta_x, delta_t, xmin, tmin, xmax, tmax, analytical_eq, gen_analytical_method, allTheTime,
    num1, num2, padding, train_takeAll, eval_takeAll, size1, size2,
    layer_data, nn_model, optim, learning_rate, iteration, hasDropout, p, weight,
    loading, load_path,
    x_file, t_file, actual_solu_file, config_file, train_loss_file, model_file, eval_loss_file, prediction_file
):
    data_setting = {'delta_x': delta_x,'delta_t': delta_t,'xmin': xmin, 'tmin': tmin, 'xmax': xmax, 'tmax': tmax, 'PDE': str(analytical_eq), 'method': str(gen_analytical_method), 'allTheTime': allTheTime}
    pairs_setting = {'left': num1,'right': num2, 'dim': num1+num2+1, 'padding': str(padding), 'train_takeAll': train_takeAll, 'eval_takeAll': eval_takeAll, 'size_train': size1, 'size_eval': size2}
    nn_setting = {'model': str(nn_model), 'layer_data': layer_data, 'input': num1+num2+1, 'output': 1, 'hasDropout': hasDropout, 'dropout_rate': p, 'res_weight': weight}
    optim_setting = {'optimizer': str(optim), 'learning_rate': learning_rate}
    model_setting = {'nn_model': nn_setting, 'optim': optim_setting, 'iteration': iteration, 'is_loading': loading, 'loading_path': load_path}
    x_file_setting = {'path': x_file, 'explain': 'x'}
    t_file_setting = {'path': t_file, 'explain': 't'}
    actual_solu_file_setting = {'path': actual_solu_file, 'explain': 'the actual/correct/analytical solutions'}
    config_file_setting = {'path': config_file, 'explain': 'the current file, store all the configurations of each experiment'}
    train_loss_file_setting = {'path': train_loss_file, 'explain': 'the training loss'}
    model_file_setting = {'path': model_file, 'explain': 'the saved model, can be loaded for further training'}
    eval_loss_file_setting = {'path': eval_loss_file, 'explain': 'the evaluation loss'}
    prediction_file_setting = {'path': prediction_file, 'explain': 'final predictions'}
    files = {
        'x_file_setting': x_file_setting,
        't_file_setting': t_file_setting,
        'actual_solu_file_setting': actual_solu_file_setting,
        'config_file_setting': config_file_setting,
        'train_loss_file_setting': train_loss_file_setting,
        'model_file_setting': model_file_setting,
        'eval_loss_file_setting': eval_loss_file_setting,
        'prediction_file_setting': prediction_file_setting
    }
    json_data = {
        'data_setting': data_setting,
        'pairs_setting': pairs_setting,
        'model_setting': model_setting,
        'files_setting': files
    }
    return json_data

In [13]:
def training_the_model(
    folder_name,
    delta_x=1/20, delta_t=1/10, xmin=0, tmin=0, xmax=1, tmax=2, analytical_eq=heat_equ_analytical_solu, gen_analytical_method=gen_analytical_cell_averaged, allTheTime=False,
    num1=3, num2=3, padding=recursive_padding, train_takeAll=True, eval_takeAll=True, size1=0, size2=0,
    layer_data=[6, 6], nn_model=FullconnectedResNet, optim=optim.Adam, learning_rate=0.001, iteration=100, hasDropout=False, p=0.2, weight=1,
    loading=False, load_path="",
    config_file="config.json", actual_solu_file='actual solution.csv', x_file='x.csv', t_file='t.csv',
    train_loss_file='training loss.txt', model_file='model',
    eval_loss_file='evaluation loss.txt', prediction_file='prediction.csv',
    doEval=True, printTraining=False, printEval=False
):
    # ====================
    # Configuration and parameters
    # ====================
    json_data = json_config_generator(
    delta_x, delta_t, xmin, tmin, xmax, tmax, analytical_eq, gen_analytical_method, allTheTime,
    num1, num2, padding, train_takeAll, eval_takeAll, size1, size2,
    layer_data, nn_model, optim, learning_rate, iteration, hasDropout, p, weight,
    loading, load_path,
    x_file, t_file, actual_solu_file, config_file, train_loss_file, model_file, eval_loss_file, prediction_file)
    # ==================
    # Creating pathes and config
    # ==================
    x_file = folder_name+"/"+x_file
    t_file = folder_name+"/"+t_file
    actual_solu_file = folder_name+"/"+actual_solu_file
    config_file = folder_name+"/"+config_file
    train_loss_file = folder_name+"/"+train_loss_file
    model_file = folder_name+"/"+model_file
    eval_loss_file = folder_name+"/"+eval_loss_file
    prediction_file = folder_name+"/"+prediction_file
    # ==========
    # Creating folder
    # ==========
    mkdir(folder_name)
    # ===========
    # Prepare the data
    # ===========
    x, t, solu = gen_analytical_method(delta_x=delta_x, delta_t=delta_t, xmin=xmin, tmin=tmin, xmax=xmax, tmax=tmax, analytical_eq=analytical_eq)
    actual_solu = solu
    if not allTheTime:
        solu = solu[:2]
    # ===========
    # Prepare the pairs
    # ===========
    if train_takeAll:
        pairs = get_trainingset_all(solu, num1, num2, padding)
    else:
        pairs = get_trainingset_random(solu, num1, num2, padding, size)
    # =================
    # Set up model & optimizer
    # =================
    model = nn_model(i=num1+num2+1, o=1, layer_data=layer_data, hasDropout=hasDropout, p=p, num=num1, weight=weight)
    if loading:
        model.load_model(load_path)
    optimizer = optim(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    # ===========
    # Train the model
    # ===========
    model.train()
    list_of_losses_t = []
    list_of_outputs = []
    counter = 0
    for itera in range(iteration):
        for pair in pairs:
            train = torch.FloatTensor(pair["train"])
            target = torch.FloatTensor([pair["target"]])
            model.zero_grad()
            optimizer.zero_grad()
            output, res, pure = model(train)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            list_of_losses_t.append(loss.item())
            list_of_outputs.append(output)
        if printTraining:
            printer_of_nn(counter=counter, train=train, target=target, loss=loss, output=output, res=res, pure=pure, TorE=True)
        counter += 1
    # for parameter in model.parameters():
    #     print(parameter)
    # =====
    # Saving
    # =====
    save_csv(actual_solu_file, actual_solu)
    save_csv(x_file, [x])
    save_csv(t_file, [t])
    save_json(config_file, json_data)
    save_list(train_loss_file, list_of_losses_t)
    model.save_model(model_file)
    # ==========
    # Do we do eval?
    # ==========
    if not doEval:
        return 0
    # ===========
    # Prepare the pairs
    # ===========
    if eval_takeAll:
        eval_pairs = get_testingset_all(actual_solu, num1, num2, padding)
    else:
        eval_pairs = get_testingset_random(actual_solu, num1, num2, padding, size)
    # =============
    # Evaluate the model
    # =============
    model.eval()
    list_of_losses_e = []
    prediction = [solu[0]]
    counter = 0
    for j in range(len(eval_pairs)):
        pairs_t = eval_pairs[j]
        prediction_t = []
        for pair in pairs_t:
            train = torch.FloatTensor(pair["train"])
            target = torch.FloatTensor([pair["target"]])
            output, res, pure = model(train)
            loss = criterion(output, target)
            list_of_losses_e.append(loss.item())
            prediction_t.append(output.item())
            if printTraining:
                printer_of_nn(counter=counter, train=train, target=target, loss=loss, output=output, res=res, pure=pure, TorE=False)
        counter += 1
        prediction.append(prediction_t)
    # =====
    # Saving
    # =====
    save_list(eval_loss_file, list_of_losses_e)
    save_csv(prediction_file, prediction)

In [ ]:
training_the_model(
    delta_x=1/10, delta_t=1/20, xmin=0, tmin=0, xmax=2, tmax=1, learning_rate=0.001, layer_data=[6], hasDropout=True, p=0.2,
    folder_name="10 10 control group [do dropout p=0.2]", iteration=1000, printTraining=True, printEval=False)

This is 1 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.1558], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.0037, grad_fn=<MseLossBackward>)
This is 2 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([0.2523], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.4081], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.1207, grad_fn=<MseLossBackward>)
This is 3 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([0.2207], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The p

The output is: tensor([-0.1558], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.0037, grad_fn=<MseLossBackward>)
This is 21 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.0569], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.0988], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.0015, grad_fn=<MseLossBackward>)
This is 22 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([0.0063], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.1621], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.01

This is 42 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.1558], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.0037, grad_fn=<MseLossBackward>)
This is 43 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.1288], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.0270], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.0011, grad_fn=<MseLossBackward>)
This is 44 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.1558], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])


The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.0355], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.0006, grad_fn=<MseLossBackward>)
This is 61 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.1311], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.0247], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.0013, grad_fn=<MseLossBackward>)
This is 62 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.1106], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.0452], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.0002, grad_fn=<MseLossBackward>)
This is 63 times of i

The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([0.0430], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.1988], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.0191, grad_fn=<MseLossBackward>)
This is 82 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.0624], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.0934], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.0011, grad_fn=<MseLossBackward>)
This is 83 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.0841], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.0717], gr

...

The output is: tensor([0.0578], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.2136], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.0234, grad_fn=<MseLossBackward>)
This is 256 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.1519], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.0039], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.0032, grad_fn=<MseLossBackward>)
This is 257 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.1558], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.

The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.1558], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.0037, grad_fn=<MseLossBackward>)
This is 278 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.1558], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.0037, grad_fn=<MseLossBackward>)
This is 279 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.1175], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.0383], grad_fn

This is 301 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.1558], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.0037, grad_fn=<MseLossBackward>)
This is 302 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.1558], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.0037, grad_fn=<MseLossBackward>)
This is 303 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.1558], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
T

This is 321 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.1558], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.0037, grad_fn=<MseLossBackward>)
This is 322 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.0954], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.0604], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(6.4883e-08, grad_fn=<MseLossBackward>)
This is 323 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.1558], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.

The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.1558], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.0037, grad_fn=<MseLossBackward>)
This is 342 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.1558], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.], grad_fn=<ReluBackward0>)
The res input is: tensor(-0.1558)
The training loss is: tensor(0.0037, grad_fn=<MseLossBackward>)
This is 343 times of iteration
Training...
The input is: tensor([-0.8873, -0.7042, -0.4521, -0.1558,  0.1558,  0.4521,  0.7042])
The output is: tensor([-0.0876], grad_fn=<AddBackward0>)
The correct solution is: tensor([-0.0951])
The pure output is: tensor([0.0682], grad_fn